<a href="https://colab.research.google.com/github/gvarnavi/generative-art-iap/blob/master/01.15-Wednesday/01_strange_attractors_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Strange Attractors
"[They're] a little like old Uncle Jake, who is a bit eccentric. You’re not really surprised by what Uncle Jake does, but it’s still difficult to understand why he does what he does. But by calling him eccentric, you feel comfortable with his actions." *~ Symmetry in Chaos, Field and Golubitsky*

---

An attractor is a state toward which a dynamical system tends to evolve, and it can take various forms such as a fixed point or a limit cycle. An attractor is called **strange** if it has a fractal structure, which often arises in chaotic systems. Although a strange attractor is locally unstable, meaning nearby points diverge from one another due to sensitive dependence on initial conditions, it is globally stable in that these points never leave the attractor. These attractive orbits can be quite... attractive!

First, let's look at some artistic trajectories that arise in discrete-time chaotic dynamical systems.

## Discrete-time systems

In [0]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import ipywidgets as widgets
from IPython.display import display
from matplotlib.colors import ListedColormap
from scipy import ndimage
from numba import jit
from google.colab import files

One popular example is the family of **De Jong attractors**, which are given by the coupled system of equations:

> $x_{n+1} = \text{sin}(ay_n) - \text{cos}(bx_n)$

> $y_{n+1} = \text{sin}(cx_n) - \text{cos}(dy_n)$

Let's write a function ```dejong``` that takes as input the current state as an array of $(x, y)$ coordinates, and a tuple of the model parameters $(a,b,c,d)$, and computes a new state array following the De Jong map.



In [0]:
@jit
def dejong(state, args):
    # unpack system parameters
    a, b, c, d = args
    
    # unpack the state vector
    x, y = state
    
    # map to the new state
    return np.array([np.sin(a*y) - np.cos(b*x),
                     np.sin(c*x) - np.cos(d*y)])

To visualize the orbit, we'll want to make a two-dimensional plot of the attractor that is colored by the number of times each point was visited over the course of many iterations. Thus, we first need a function, ```iter_map```, to iterate our discrete map a specified number of times starting from a particular initial state.

In [0]:
@jit
def iter_map(func, steps, s0, args):
    ''' Iterates the map given by func over a specified number of steps
        starting from initial state s0. Additional arguments passed to func
        are given in args.
    '''
    sol = np.zeros((steps, len(s0)))
    sol[0,:] = s0
    for i in range(1,steps):
        sol[i,:] = func(sol[i-1,:], args)
    return sol.T

The generated states (coordinates) will generally be continuous numbers, but to form an image colored by visitation density, we'll need to bin the points into discrete image pixels. Thus, we define a function ```grid_data``` to bin the continuous $(x, y)$ coordinates generated by the running ```iter_map``` into discrete image pixels.

In [0]:
def grid_data(x, y, nx, ny):
    ''' Map continuous (x, y) coordinate data to a discrete grid. '''
    xmin, xmax, ymin, ymax = np.min(x), np.max(x), np.min(y), np.max(y)
    x = ((x - xmin)/(xmax - xmin)*nx - xmin).astype(np.int)
    y = ((y - ymin)/(ymax - ymin)*ny - ymin).astype(np.int)
    return x, y

We want to color our pixels based on visitation density. To do this, we define a couple of helper functions. The function ```make_figure``` packages some plotting commands to create and style our figure axes and select a colormap. The function ```color_by_number``` is going to take our list of gridded data and associate a color with each grid point based on its occupancy. Because the occupancy data tend to be skewed, this function is not a linear map between color and data, but performs a scaling of the data that allows it to more equitably sample the colorspace.

In [0]:
def make_figure():
    ''' Set-up figure and colormap. '''
    f = plt.figure(figsize=(7,7))
    ax = f.add_subplot()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    cmap = plt.get_cmap('CMRmap')
    ax.set_facecolor('#070215')
    return f, ax, cmap

def color_by_number(x, y, cmap):
    ''' Set the color of data points based on visitation density. '''
    inverse, counts = np.unique(x + y*1j, return_counts=True, return_inverse=True)[1:]
    counts = counts[inverse]/np.max(counts)
    
    # scale and standardize data to more equitably sample colorspace
    counts = counts**0.05
    counts = (counts - np.min(counts))/(np.max(counts)-np.min(counts)+1e-3)
    counts[counts > 1.8*np.mean(counts)] = 1.8*np.mean(counts)
    counts = (counts - np.min(counts))/(np.max(counts)-np.min(counts))
    
    colors = 250*cmap(counts)
    return colors

Finally, we'll define a function ```make_map``` to color each image pixel the desired color.

In [0]:
@jit
def make_map(image, x, y, colors):
    ''' Color image pixels a specified color. '''
    for k in range(len(x)):
        image[x[k],y[k],:] = colors[k]
    return image

We can package all these steps into a function ```plot_image```. This will be our helper function to repurpose many times for various two-dimensional systems, so it is written as generically as possible.

In [0]:
def plot_image(func, s0, args):
    ''' Plot an image colored by the point's visitation density. '''
    # evaluate map over many iterations
    N = int(5e6)                                  # number of steps to take
    sol = iter_map(func, N, s0, args)

    #ignore transient part of solution space
    sol = sol[:,2000:]
    x, y = sol[0,:], sol[1,:]

    # initialize image array
    nx = 1000
    ny = 1000
    image = np.zeros((nx+1, ny+1, 4), dtype=np.uint8)
    
    # format data to grid
    x, y = grid_data(x, y, nx, ny)

    # initialize axes and colormap
    f, ax, cmap = make_figure()

    # assign colors based on number of times point was visited
    colors = color_by_number(x, y, cmap)

    # color image pixels
    make_map(image, x, y, colors)

    # create a border by padding with some zeros
    npad = 100
    image = np.pad(image, ((npad,npad), (npad,npad), (0,0)), mode='constant')

    # smooth image a little
    image = ndimage.uniform_filter(image, size=(5,5,0))

    # plot
    ax.imshow(np.flipud(image))
    return f, ax

Now let's plot the De Jong attractor! Test some of the suggested parameters below. Although the code is fairly optimized, we are plotting millions of points so there may be a slight delay each time you plot.

In [0]:
@widgets.interact_manual(args = ['leaf','whale','cocoon','ribbon','crescent','squid','heart'])
def dejong_dropdown(args='leaf'):
    # suggested parameter sets
    leaf = (-2., -2., -1.2, 2.)
    whale = (2.01, -2.53, 1.61, -0.33)
    cocoon = (-2.850, 2.793, -2.697, 1.128)
    ribbon = (1.5, 2.5, 0.731, 2.5)
    crescent = (1.549, 1.104, 2.4, -2.1)
    squid = (-1.33, -2., -1.2, 2.)
    heart = (2.03, -2., -1.2, 2.)

    plot_image(dejong, np.array([-0.3, 0.2]), eval(args))

Try discovering your own parameters for a unique piece of abstract art. Remember that different parameter windows can produce strikingly different limiting behaviors, so try tuning one parameter at a time to avoid running into too many seemingly blank images! If you get stuck, you can always re-run the cell to reset the defaults, or copy the values of a suggested parameter set above and proceed from there.

In [0]:
a_slider = widgets.FloatSlider(value=-2., min=-3., max=3., step=0.01, description='a:', readout_format='.2f')
b_slider = widgets.FloatSlider(value=-2., min=-3., max=3., step=0.01, description='b:', readout_format='.2f')
c_slider = widgets.FloatSlider(value=-1.2, min=-3., max=3., step=0.01, description='c:', readout_format='.2f')
d_slider = widgets.FloatSlider(value=2., min=-3., max=3., step=0.01, description='d:', readout_format='.2f')

@widgets.interact_manual(a = a_slider, b = b_slider, c = c_slider, d = d_slider)
def dejong_slider(a=-2., b=-2., c=-1.2, d=2.):
    plot_image(dejong, np.array([-0.3, 0.2]), (a, b, c, d))

To plot a static image with a fixed set of parameters and avoid using the widget, you can manually enter the parameter values you'd like (or copy and paste some from the list above) and then execute the following cell:

In [0]:
# manually enter the 4 parameter values into the variable args
# parameters should be comma separated and enclosed in parentheses as shown
args = (-2., -2., -1.2, 2.)

# plot the image
plot_image(dejong, np.array([-0.3, 0.2]), args)
plt.show()

### Saving Figures

After we've found an interesting set of parameters, we can use the ```savefig``` command to save our figure following the syntax below. Make sure to change im_name to the desired image file name, and to change the parameter values to the ones you found! Afterward, we will run one more cell to download the image from the google colab files.

In [0]:
#*********************** change me **************************#
# input the name you want to give your image file as a string
im_name = 'my_image'

# enter the parameter values
args = (2.03, -2., -1.2, 2.)
#************************************************************#

# plot the image outside of the widget and retain the figure handle f
f, ax = plot_image(dejong, np.array([-0.3, 0.2]), args)

# save image
f.savefig(im_name + '.png')

If you are working from a local copy of this notebook, the saved image should appear in the same directory as this notebook. If you are not running a local copy of the notebook, but are working in the Google Colaboratory environment, you will need to download your image after saving it by executing the command below. If you get an error when downloading, the image was probably not fully finished saving, so just wait a few seconds and then run the download command again. 

In [0]:
# download image
files.download(im_name + '.png')

The image should then download into your Downloads folder.

Another interesting example is the family of **Clifford attractors**, which are given by the coupled system of equations:

> $x_{n+1} = \text{sin}(ay_n) - c\text{cos}(ax_n)$

> $y_{n+1} = \text{sin}(bx_n) - d\text{cos}(by_n)$

Let's write a function ```clifford``` to compute the new state array following the Clifford map.

In [0]:
@jit
def clifford(state, args):
    # unpack system parameters
    a, b, c, d = args
    
    # unpack the state vector
    x, y = state
    
    # map to the new state
    return np.array([np.sin(a*y) - c*np.cos(a*x),
                     np.sin(b*x) - d*np.cos(b*y)])

Now let's plot as before using our visualization code! Try some of the suggested parameters.

In [0]:
@widgets.interact_manual(args = ['arcs','shell','roll','squiggle'])
def clifford_dropdown(args='leaf'):
    # suggested parameter sets
    arcs = (-1.4, 1.6, 1., 0.7)
    shell = (1.7, 1.7, 0.6, 1.2)
    roll = (1.5, -1.8, 1.6, 0.9)
    squiggle = (-1.8, -2.0, -0.5, -0.9)

    plot_image(clifford, np.array([-0.3, 0.2]), eval(args))

Play around with the sliders to try and find some other abstract designs.

In [0]:
a_slider = widgets.FloatSlider(value=-1.4, min=-2., max=2., step=0.01, description='a:', readout_format='.2f')
b_slider = widgets.FloatSlider(value=1.6, min=-2., max=2., step=0.01, description='b:', readout_format='.2f')
c_slider = widgets.FloatSlider(value=1., min=-2., max=2., step=0.01, description='c:', readout_format='.2f')
d_slider = widgets.FloatSlider(value=0.7, min=-2., max=2., step=0.01, description='d:', readout_format='.2f')

@widgets.interact_manual(a = a_slider, b = b_slider, c = c_slider, d = d_slider)
def clifford_slider(a=-1.4, b=1.6, c=1., d=0.7):
    plot_image(clifford, np.array([-0.3, 0.2]), (a, b, c, d))

Static plotting option:

In [0]:
# manually enter the 4 parameter values into the variable args
# parameters should be comma separated and enclosed in parentheses as shown
args = (-1.4, 1.6, 1., 0.7)

# plot the image
plot_image(clifford, np.array([-0.3, 0.2]), args)
plt.show()

Optionally save your image:

In [0]:
#*********************** change me **************************#
# input the name you want to give your image file as a string
im_name = 'my_image'

# enter the parameter values
args = (-1.4, 1.6, 1., 0.7)
#************************************************************#

# plot the image outside of the widget and retain the figure handle f
f, ax = plot_image(clifford, np.array([-0.3, 0.2]), args)

# save image
f.savefig(im_name + '.png')

In [0]:
# download image
files.download(im_name + '.png')

### Symmetry

*Adapted from Shaw, William T. Complex Analysis with Mathematica. Cambridge University Press, 2006.*

Extraordinary patterns can arise when chaotic systems are constructed to obey some underlying symmetry. The simplest examples are complex polynomial maps that are symmetric under the actions of the **cyclic group** $Z_n$ of rotations by $2 \pi/n$, or the **dihedral group** $D_n$, consisting of rotations by $2 \pi/n$ and a reflection. Note that this $n$ is different from the $n$ we have been using to denote our iteration steps, so we will let $k$ denote our iteration steps to avoid confusion!

The general functional form of such maps is given by:

$z_{k+1} = f(z_k) = z_k(a + bz_k\bar{z_k} + c\text{Re}(z_k^n) +i\omega) + d\bar{z_k}^{n-1}$

with $a,b,c,d,\omega$ real and $n$ an integer. These are cyclically symmetric and inherit dihedral symmetry when $\omega = 0$.

Below, we implement a slight variant of this form which we term the nonlinear map:

$f_\text{nonlinear}(z_k) = z_k(a + bz_k\bar{z_k} + c\text{Re}(z_k^n) + \lambda |z_k| \text{cos}(\text{arg}(z_k)np) +i\omega) + d\bar{z_k}^{n-1}$

where arg denotes the argument (angle) of a complex number, and with $a,b,c,d,\lambda, \omega$ real and $n, p$ integers.

In [0]:
@jit
def nonlinear(state, args):
    # specify system parameters
    a, b, c, d, l, w, n, p = args

    # unpack the state vector
    x, y = state
    z = complex(x, y)
    
    # map to the new state
    F = (a + b*z*np.conj(z) + c*(z**n).real + l*np.abs(z)*np.cos(np.angle(z)*n*p) + w*1j)*z + d*np.conj(z)**(n-1)
    return np.array([F.real, F.imag])

Now let's visualize! Try out some of the suggested parameter sets for the different symmetries.

In [0]:
@widgets.interact_manual(args = ['threegadget','triangle','quadgig','flint','pentagon','star','sanddollar',
                                 'flower','hexagon','heptagon','churwin','swirlygig','bloom'])
def nonlinear_dropdown(args='sanddollar'):
    # dihedral symmetry parameter sets
    threegadget = (1.56, -1., 0.1, -0.82, 0., 0., 3, 1)
    triangle = (1.455, -1.0, 0.03, -0.8, -0.025, 0., 3, 0)
    quadgig = (-1.86, 2., 0., 1., 0., 0., 4, 0)
    flint = (2.5, -2.5, 0., 0.9, 0., 0., 3, 1)
    pentagon = (2.6, -2., 0., -0.5, 0., 0., 5, 1)
    star = (-2.32, 2.32, 0., 0.75, 0., 0., 5, 1)
    sanddollar = (-2.34, 2., 0.2, 0.1, 0., 0., 5, 1)
    flower = (-2.38, 10.0, -12.3, 0.75, 0.02, 0., 5, 1)
    hexagon = (-2.7, 5., 1.5, 1., 0., 0., 6, 1)
    heptagon = (-2.08, 1.0, -0.1, 0.167, 0., 0., 7, 1)
    churwin = (2.409, -2.5, -0.2, 0.81, 0., 0., 24, 1)

    # cyclical symmetry parameter sets
    swirlygig = (-1.86, 2., 0., 1., 0., 0.1, 4, 0)
    bloom = (-2.5, 5., -1.9, 1., 0., 0.188, 5, 0)

    plot_image(nonlinear, np.array([-0.3, 0.2]), eval(args))

Static plotting option:

In [0]:
# manually enter the 8 parameter values into the variable args
# parameters should be comma separated and enclosed in parentheses as shown
args = (-2.34, 2., 0.2, 0.1, 0., 0., 5, 1)

# plot the image
plot_image(nonlinear, np.array([-0.3, 0.2]), args)
plt.show()

Optionally save your image:

In [0]:
#*********************** change me **************************#
# input the name you want to give your image file as a string
im_name = 'my_image'

# enter the parameter values
args = (-2.34, 2., 0.2, 0.1, 0., 0., 5, 1)
#************************************************************#

# plot the image outside of the widget and retain the figure handle f
f, ax = plot_image(nonlinear, np.array([-0.3, 0.2]), args)

# save image
f.savefig(im_name + '.png')

In [0]:
# download image
files.download(im_name + '.png')

Another interesting family of symmetric maps may be constructed with periodic functions like sinusoids. Below, we implement one variant:

> $x_{n+1} = mx_n + a\text{sin}(2 \pi x_n) + b\text{sin}(2 \pi x_n)\text{cos}(2 \pi y_n) + c \text{sin}(4 \pi x_n) + d \text{sin}(6 \pi x_n) \text{cos}(4 \pi y_n)$

> $y_{n+1} = my_n + a\text{sin}(2 \pi y_n) + b\text{cos}(2 \pi x_n)\text{sin}(2 \pi y_n) + c \text{sin}(4 \pi y_n) + d \text{cos}(4 \pi x_n) \text{sin}(6 \pi y_n)$

In [0]:
@jit
def periodic(state, args):
    # specify system parameters
    a, b, c, d, m = args

    # unpack the state vector
    x, y = state
    
    # map to the new state
    return np.array([m*x + a*np.sin(2.*np.pi*x) + b*np.sin(2.*np.pi*x)*np.cos(2.*np.pi*y)
                     + c*np.sin(4.*np.pi*x) + d*np.sin(6.*np.pi*x)*np.cos(4.*np.pi*y),
                     m*y + a*np.sin(2.*np.pi*y) + b*np.cos(2.*np.pi*x)*np.sin(2.*np.pi*y)
                     + c*np.sin(4.*np.pi*y) + d*np.cos(4.*np.pi*x)*np.sin(6.*np.pi*y)])

We can take advantage of the built-in periodicity to produce beautiful tiled patterns. The function ```tile_image``` repurposes most of our existing code, but further creates a quilt-like pattern of smaller images.

In [0]:
def tile_image(func, s0, args, reps):
    ''' Plot a tiled image colored by the point's visitation density. '''
    # evaluate map over many iterations
    N = int(5e6)                                  # number of steps to take
    sol = iter_map(func, N, s0, args)

    #ignore transient part of solution space
    sol = sol[:,2000:]
    x, y = sol[0,:], sol[1,:]

    # initialize image array
    nx = 1000
    ny = 1000
    nx_tile = nx//reps
    ny_tile = ny//reps
    image = np.zeros((nx_tile+1, ny_tile+1, 4), dtype=np.uint8)
    
    # format data to grid
    x, y = grid_data(x, y, nx_tile, ny_tile)
    
    # initialize axes and colormap
    f, ax, cmap = make_figure()

    # assign colors based on number of times point was visited
    colors = color_by_number(x, y, cmap)

    # color image pixels
    make_map(image, x, y, colors)
    
    # tile into a quilt
    image = np.tile(image, (reps, reps, 1))
    
    # create a border by padding with some zeros
    npad = 100
    image = np.pad(image, ((npad,npad), (npad,npad), (0,0)), mode='constant')

    # smooth image a little
    image = ndimage.uniform_filter(image, size=(5,5,0))

    # plot
    ax.imshow(np.flipud(image))
    return f, ax

Let's plot.

In [0]:
@widgets.interact_manual(args = ['clover','flower','plus','plus2','plate','cross'], reps = [2,3,4])
def periodic_dropdown(args='clover', reps=3):
    # suggested parameter sets
    clover = (0.25, -0.3, 0.2, 0.3, 1.)
    flower = (0.25, -0.3, 0.1, 0.3, 1.)
    plus = (0.2, -0.2, 0.2, 0.3, 1.)
    plus2 = (0.2, -0.25, 0.2, 0.3, 1.)
    plate = (0.2, -0.2, 0., 0.3, 1.)
    cross = (0.2, -0.2, 0., 0.3, 0.)

    tile_image(periodic, np.array([-0.3, 0.2]), eval(args), reps)

Static plotting option:

In [0]:
# manually enter the 5 parameter values into the variable args
# parameters should be comma separated and enclosed in parentheses as shown
args = (0.25, -0.3, 0.2, 0.3, 1.)

# enter number of tiles per side
reps = 3

# plot the image
tile_image(periodic, np.array([-0.3, 0.2]), args, reps)
plt.show()

Optionally save your image:

In [0]:
#*********************** change me **************************#
# input the name you want to give your image file as a string
im_name = 'my_image'

# enter the parameter values
args = (0.25, -0.3, 0.2, 0.3, 1.)
reps = 3
#************************************************************#

# plot the image outside of the widget and retain the figure handle f
f, ax = tile_image(periodic, np.array([-0.3, 0.2]), args, reps)

# save image
f.savefig(im_name + '.png')

In [0]:
# download image
files.download(im_name + '.png')

## Continuous-time systems

Chaotic dynamical systems parameterized by continuous time make for interesting animations! And because chaotic systems are characterized by a sensitive dependence on intital conditions, the possible trajectories can diverge rapidly even when the starting conditions differ only slightly. To investigate these properties, we will look at some famous three-dimensional chaotic systems and animate their evolution in time.

First, we need some additional imports for animation and 3D plotting.

In [0]:
import matplotlib.animation as animation
from scipy.integrate import solve_ivp
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from IPython.display import HTML

One very well known chaotic attractor is the **Lorenz attractor**, which arises from the evolution of the differential equations:

> $x' = a(y - x)$

> $y' = x(b - z) - y$

> $z' = xy - cz$

We will implement a function ```lorenz``` to evaluate these functions.




In [0]:
@jit
def lorenz(t, state, args):
    # unpack system parameters
    a, b, c = args
    
    # unpack the state vector
    x, y, z = state
    
    # evaluate system of differential equations at new state
    return np.array([a*(y - x),
                     x*(b - z) - y,
                     x*y - c*z])

Note that in contrast to discrete systems, we now have a function that gives us not the next state, but the local time derivative that tells us how to evolve the system. To solve the differential equations, we will use the function ```solve_ivp``` from the ```scipy``` library. Let's define a function ```solve_system``` to format our input conditions and output a list of three arrays $x, y, z$ of the system's trajectory.

In [0]:
def solve_system(system_name, N, tr, params):
  ''' Solve selected continuous-time system over time. '''
  mapfun, s0, tf, args = eval(system_name), *params
  
  # time points at which to evaluate
  t = np.linspace(0, tf, N)

  # solve using scipy solver
  sol = solve_ivp(lambda t, s: mapfun(t, s, args), (0, tf), s0, t_eval=t)

  x, y, z = sol.y[0,tr:], sol.y[1,tr:], sol.y[2,tr:]
  return [x, y, z]

Finally, we want to define a few helper functions that will allow us to make colored plots of the trajectories in three dimensions.

In [0]:
def make_segments3D(x, y, z):
    points = np.array([x, y, z]).T.reshape(-1, 1, 3)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    return segments

def colorline3D(ax, x, y, z, cmap):
    N = len(x)
    norm = plt.Normalize(0,0.9)
    segments = make_segments3D(x, y, z)
    lc = Line3DCollection(segments, array=np.linspace(0,1,N), cmap=cmap, norm=norm, linewidth=1.5)
    return lc

def set_limits3D(ax, x, y, z):
    ax.set_xlim([np.min(x), np.max(x)])
    ax.set_ylim([np.min(y), np.max(y)])
    ax.set_zlim([np.min(z), np.max(z)])

def make_figure3D():
    ''' Set-up figure and colormap. '''
    f = plt.figure(figsize=(7,7))
    ax = f.add_subplot(projection='3d')
    ax.axis('off')
    f.tight_layout()

    cmap = plt.get_cmap('magma')
    return f, ax, cmap

Now we are ready to plot!

In [0]:
system_name = 'lorenz'

# initial state, final time, model parameters
lorenz_params = np.array([1.,1.,1.]), 40., (10., 28., 8./3.)

# solve for the trajectories
N = 10000                                     # number of time steps to take
tr = 100                                      # discard transient steps
x, y, z = solve_system(system_name, N, tr, eval(system_name + '_params'))

# plot
f, ax, cmap = make_figure3D()
ax.add_collection3d(colorline3D(ax, x, y, z, cmap))
set_limits3D(ax, x, y, z)
plt.show()

Let's implement another famous example, the **Rossler attractor**. The Rossler system is given by:
> $x' = -y - z$

> $y' = x + ay$

> $z' = b + z(x - c)$

In [0]:
@jit
def rossler(t, state, args):
    # unpack system parameters
    a, b, c = args
    
    # unpack the state vector
    x, y, z = state
    
    # evaluate system of differential equations at new state
    return np.array([-y - z,
                     x + a*y,
                     b + z*(x - c)])

We can test it out by updating our earlier code with the Rossler attractor parameters and changing the system name.

In [0]:
system_name = 'rossler'

# initial state, final time, model parameters
lorenz_params = np.array([1.,1.,1.]), 40., (10., 28., 8./3.)
rossler_params = np.array([1.,1.,0.]), 200., (0.1, 0.1, 14.)

# solve for the trajectories
N = 10000                                     # number of time steps to take
tr = 100                                      # discard transient steps
x, y, z = solve_system(system_name, N, tr, eval(system_name + '_params'))

# plot
f, ax, cmap = make_figure3D()
ax.add_collection3d(colorline3D(ax, x, y, z, cmap))
set_limits3D(ax, x, y, z)
plt.show()

Now let's make an animation of the output trajectories. Note that because we have to keep plotting more and more segments in each frame, the animation may take a minute to run.

In [0]:
system_name = 'lorenz'

# initial state, final time, model parameters
lorenz_params = np.array([1.,9.,-7.]), 15., (10., 28., 8./3.)
rossler_params = np.array([1.,-1.,5.]), 80., (0.1, 0.1, 14.)

# solve for the trajectories
N = 2100                                    # number of time steps to take
tr = 100                                    # discard transient steps
x, y, z = solve_system(system_name, N, tr, eval(system_name + '_params'))

# initialize plot
f, ax, cmap = make_figure3D()
ax.view_init(30,0)
set_limits3D(ax, x, y, z)

# starting frame
line = colorline3D(ax, x[:2], y[:2], z[:2], cmap)
def init():
    line.set_array(np.linspace(0, 2./N, 1))
    ax.add_collection3d(line)
    return line,

# animation
spf = 4                                     # time steps per frame
def animate(i):
    # step two time-steps per frame
    i = (spf*i)%(N-tr)
    line.set_segments(make_segments3D(x[1:i], y[1:i], z[1:i]))
    line.set_array(np.linspace(1./N, i/N, i))
    ax.view_init(30, 0.6*i/spf)
    return line,

ani = animation.FuncAnimation(f, animate, init_func=init, frames=(N-tr)//spf, interval=30, blit=True)
plt.close(f)

# display animation
HTML(ani.to_html5_video())

### Sensitive dependence on initial conditions

Let's make an animation that reveals how the system evolves given slightly different initial conditions. The animation code below was adapted from [this blog](https://jakevdp.github.io/blog/2013/02/16/animating-the-lorentz-system-in-3d/), which includes a number of other physics-inspired Python coding challenges you may want to try!

In [0]:
system_name = 'rossler'

# final time, model parameters
lorenz_params = 4., (10., 28., 8./3.)
rossler_params = 20., (0.1, 0.1, 14.)

# initialization parameters
L = 12                      # a random starting point will be generated on [-L,L]
N_paths = 20                # number of different initial conditions to test                                                
np.random.seed(12)
s0 = -L + 2*L*np.random.random((N_paths, 3))
N = 1000                    # number of time steps to take
tr = 0                      # don't discard any transient steps
sol = [[]]*N_paths

for i, si in enumerate(s0):
    params = (si, *eval(system_name + '_params'))
    sol[i] = solve_system(system_name, N, tr, params)

f, ax, _ = make_figure3D()
ax.view_init(30,0)

# set limits
lims = [[]]*3
for s in sol:
    lims[0] += [np.min(s[0]), np.max(s[0])]
    lims[1] += [np.min(s[1]), np.max(s[1])]
    lims[2] += [np.min(s[2]), np.max(s[2])]

lims = np.array(lims)
set_limits3D(ax, 0.7*lims[0], 0.7*lims[1], 0.7*lims[2])

# choose a different color for each trajectory
colors = plt.get_cmap('rainbow')(np.linspace(0, 1, N_paths))

# initialize lines and points artists for each trajectory
lines = sum([ax.plot([], [], [], '-', c=c) for c in colors], [])
markers = sum([ax.plot([], [], [], 'o', c=c) for c in colors], [])
def init():
    for l, m in zip(lines, markers):
        l.set_data([], [])
        l.set_3d_properties([])
        m.set_data([], [])
        m.set_3d_properties([])
    return lines + markers

# animation
spf = 2             # time steps per frame
def animate(i):
    i = (spf*i)%len(sol[0][0])
    for l, m, s in zip(lines, markers, sol):
        x, y, z = s[0][:i], s[1][:i], s[2][:i]
        l.set_data(x, y)
        l.set_3d_properties(z)
        m.set_data(x[-1:], y[-1:])
        m.set_3d_properties(z[-1:])

    ax.view_init(30, 0.6*i/spf)
    return lines + markers

ani = animation.FuncAnimation(f, animate, init_func=init, frames=(N-tr)//spf, interval=30, blit=True)
plt.close(f)

# display animation
HTML(ani.to_html5_video())